In [1]:
!pip install pytorch_lightning
!pip install timm
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 라이브러리 호출

학습에 필요한 라이브러리르 호출한다.

In [2]:
import timm
import random
import os

import pandas as pd
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torchvision import transforms

import albumentations as A
import albumentations.pytorch

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from einops import rearrange, reduce, repeat

import gc

import warnings
warnings.filterwarnings(action='ignore') 

## 구글 드라이브 연결

구글 코랩에서 학습을 수행하기 위해서는 기본적으로 코랩에 연결시켜야 한다. 아래와 같이 수행하면 된다.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip "/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/datasets/open.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/TRAIN_71888.png   
  inflating: train/TRAIN_71889.png   
  inflating: train/TRAIN_71890.png   
  inflating: train/TRAIN_71891.png   
  inflating: train/TRAIN_71892.png   
  inflating: train/TRAIN_71893.png   
  inflating: train/TRAIN_71894.png   
  inflating: train/TRAIN_71895.png   
  inflating: train/TRAIN_71896.png   
  inflating: train/TRAIN_71897.png   
  inflating: train/TRAIN_71898.png   
  inflating: train/TRAIN_71899.png   
  inflating: train/TRAIN_71900.png   
  inflating: train/TRAIN_71901.png   
  inflating: train/TRAIN_71902.png   
  inflating: train/TRAIN_71903.png   
  inflating: train/TRAIN_71904.png   
  inflating: train/TRAIN_71905.png   
  inflating: train/TRAIN_71906.png   
  inflating: train/TRAIN_71907.png   
  inflating: train/TRAIN_71908.png   
  inflating: train/TRAIN_71909.png   
  inflating: train/TRAIN_71910.png   
  inflating: train/TRAIN_71911.png   
  inflating: train/TRAIN_71912.png   
  inflating: t

## 시드값 고정

아래와 같이 시드값을 고정해야지 매번 학습할 때 마다 동일한 결과를 얻을 수 있다. 시드 값은 하고 싶은 숫자를 하면 되고 저는 생일로 하였습니다 😊.

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(428)

## 학습 데이터를 불러옴

수정

In [6]:
train_df = pd.read_csv('train.csv')

In [7]:
# 학습 데이터로부터 단어 사전(Vocabulary) 구축
train_gt = [gt for gt in train_df['label']]
train_gt = "".join(train_gt)
letters = sorted(list(set(list(train_gt))))
print(len(letters))

2349


In [8]:
vocabulary = ["-"] + letters
print(len(vocabulary))
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {v:k for k,v in idx2char.items()}

2350


In [9]:
train_df.head()

,id,img_path,label
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색
1,TRAIN_00001,./train/TRAIN_00001.png,머
2,TRAIN_00002,./train/TRAIN_00002.png,차차
3,TRAIN_00003,./train/TRAIN_00003.png,써
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다


In [10]:
train_df['label_len'] = train_df['label'].apply(lambda x: len(x))

In [11]:
train_df.head()

,id,img_path,label,label_len
0,TRAIN_00000,./train/TRAIN_00000.png,빨간색,3
1,TRAIN_00001,./train/TRAIN_00001.png,머,1
2,TRAIN_00002,./train/TRAIN_00002.png,차차,2
3,TRAIN_00003,./train/TRAIN_00003.png,써,1
4,TRAIN_00004,./train/TRAIN_00004.png,놓치다,3


In [12]:
import torchmetrics

In [13]:
from torch.nn.utils.rnn import pad_sequence

transform_train = A.Compose(
    [
        # A.RandomResizedCrop(
        #     height=128, 
        #     width=256, 
        #     scale=(0.24, 0.26),
        #     ratio=(0.90, 1.10),
        #     always_apply=True
        #     ),
        A.Resize(64, 128),
        A.VerticalFlip(p=0.5),
        albumentations.OneOf([
                            albumentations.MotionBlur(p=0.5),
                            albumentations.OpticalDistortion(p=0.5),
                            albumentations.GaussNoise(p=0.5)                 
        ], p=1),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.transforms.ToTensorV2()
        ])

transform_test = A.Compose(
    [
        A.Resize(64, 128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.pytorch.transforms.ToTensorV2()
        ])

"""
    지금에서야 깨달음
    Collator로 넘어오기 전에 Dataset에서 처리가 다 끝나고 batch_size 만큼
    리스트로해서 Collator에게 넘겨주네 ...
    그러니 DataLoader에서 너가 선택한 batch_size 만큼 수정을 해줘야함.
"""
class TextCollator():
    def __init__(self, is_train = False):
        self.is_train = is_train
    
    def __call__(self, samples):
        if self.is_train:
            return_image = []
            return_label = [torch.LongTensor([0,0,0,0,0,0])] # pad_sequence 용으로 처음에는 하나를 미리 넣어준다. 그래야 출력 길이가 같아지기 때문이다.
            for i in range(len(samples)):
                image, label = samples[i]
                return_image.append(image.unsqueeze(0))
                return_label.append(torch.LongTensor([char2idx[x] for x in label]))
            # print("😁")
            return_image = torch.vstack(return_image)
            return_label = pad_sequence(return_label, batch_first = True)
            return_label = torch.LongTensor(return_label)
            return_label = return_label[1:, :]
            # |reutrn_label| = (batch_size, 6)
            # print("return_image ", return_image.shape)
            # print("return_label", return_label.shape)
            # print(return_label)
            # print("😂")
            return return_image, return_label
        else:
            return_image = []
            for i in range(len(samples)):
                image = samples[i]
                return_image.append(image.unsqueeze(0))
            return_image = torch.vstack(return_image)
            return return_image

class TextDataset(Dataset):
    def __init__(self, images, labels = None, is_train = False, is_valid = False):
        self.images = images
        self.labels = labels
        self.is_train = is_train
        self.is_valid = is_valid
    def __len__(self):
        return len(self.images)

    """
        is_train : 학습에서는 이미지 변경을 수행해주는 작업이 필요하기 때문에
        학습 중이라는 별도의 표시가 필요함
        is_valid : 학습에서는 이미지 변경이 필요하지만 검증에서는 불필요 하기
        때문에 별도로 빼서 진행
    """
    def __getitem__(self, idx):
        image_path = self.images[idx]
        if self.is_train or self.is_valid:
            label = self.labels[idx]
        temp = Image.open(image_path).convert("RGB")
        image = np.array(temp).copy()
        temp.close()

        if self.is_train:
            # 학습 데이터
            transformed = transform_train(image = image)
            image = transformed['image']
            return (image, label)
        elif self.is_valid:
            # 검증 데이터 
            transformed = transform_test(image = image)
            image = transformed['image']
            return (image, label)
        else:
            # 테스트 데이터
            transformed = transform_test(image = image)
            image = transformed['image']
            return image

In [14]:
config = {
    'n_splits' : 10,
    'random_seed' : 428,
    'batch_size' : 256,
    'input_size' : 1024,
    'hidden_size' : 1024,
    'num_layers' : 1,
    'dropout' : 0.1,
    'model' : 'regnety_160',
    'vocabulary_size' : len(vocabulary),
    'accumulate_grad_batches' : 4,
    'patience' : 5,
    'max_epochs' : 100
}

In [15]:
# eff = timm.create_model(config['model'], pretrained=True)
# # CNN Feature Extract
# eff = list(eff.children())[:-2]
# feature_extract = nn.Sequential(
#     *eff
# )
# train_dataset = TextDataset(train_df['img_path'].reset_index(drop=True), train_df['label'].reset_index(drop=True), is_train = True)
# images, labels = train_dataset[0]
# feature_extract(images.unsqueeze(0)).shape

In [16]:
# assert False

In [17]:
# eff =  timm.create_model(config['model'], pretrained=True)
# eff_feature_extract = nn.Sequential(*list(eff.children())[:-3]) 
# eff_linear = nn.Sequential(
#     nn.Linear(1232 * 4, 1024, bias = False),
# )

In [18]:
# image = torch.rand(size=(32, 3, 64, 128))

In [19]:
# eff_feature_extract(image).shape

In [20]:
from pytorch_lightning.accelerators import accelerator
class OCRModel(pl.LightningModule):
    def __init__(self, config):
        # Regnetx_006 -> torch.Size([1, 528, 4, 8]) [:-2]
        # Regnetx_032 -> torch.Size([1, 1008, 4, 8]) [:-2]
        # Effnet -> # torch.Size([2, 512, 4, 8]) [:-4]
        # regnety_160 -> torch.Size([32, 1232, 4, 8])
        super().__init__()
        self.config = config
        eff =  timm.create_model(config['model'], pretrained=True)
        self.eff_feature_extract = nn.Sequential(*list(eff.children())[:-3]) 
        self.eff_linear = nn.Sequential(
            nn.Linear(1232 * 4, 1024, bias = False),
        )
        # -> batch_size x 8 x (1232 * 4)
        
        self.lstm = nn.LSTM(input_size = config['input_size'], # input_size = 1024
                            hidden_size = config['hidden_size'], #  hiddden_size = 1024
                            num_layers = config['num_layers'],  # num_laters 
                            dropout = config['dropout'], # dropout
                            bidirectional = True,
                            batch_first = True)
        
        # |lstm| = (batch_size, 8, 1024 * 2)

        self.three_linear = nn.Linear(2048*3, config['vocabulary_size'])
        self.two_linear = nn.Linear(2048*2, config['vocabulary_size'])

        loss_weight = torch.ones(config['vocabulary_size'])
        # loss_weight[0] = 0

        self.criterion = nn.CrossEntropyLoss(
            weight = loss_weight
        )

    def forward(self, x):
        pass

    def get_acc(self, y_hat, y):
        # |y_hat| = (batch_size, 6, vocabulary_size)
        # |y| = (batch_size, 6)
        y_hat = y_hat.argmax(dim = -1)
        # |y_hat| = (batch_size, 6)

        y_hat_str = []
        y_str= []

        for i in range(y_hat.shape[0]):
            y_hat_str.append(''.join([str(int(x)) for x in y_hat[i]]))
            y_str.append(''.join([str(int(x)) for x in y[i]]))

        acc = accuracy_score(y_hat_str, y_str)
        
        del y_hat_str, y_str
        gc.collect()

        return acc

    def compute_loss(self, y_hat, y, is_valid=False): # labels, context
        # |y_hat| = (batch_size, 6, vocabulary_size)
        # llabels| = (batch_size, 6)

        acc = None
        # acc구할 차례
        if is_valid:
            acc = self.get_acc(y_hat, y)

        loss = self.criterion(y_hat.view(-1, self.config['vocabulary_size']), y.view(-1))            

        return loss, acc

    def training_step(self, batch, batch_idx):
        # batch = (image, label)
        # |image| = (batch_size, channel, h, w)
        # <수정전>
        # |label| = ("안녕하세요", "나는") <- tuple 형태로 담겨져 있음
        # <수정후>
        # |label| = (batch_size, 6)
        images, labels = batch
        batch_size = images.shape[0]

        representation = self.eff_feature_extract(images)
        # |representation| = (batch_size,1232, 4, 8) = (batch_size, channel, height, width)
        representation = representation.permute(0, 3, 1, 2)
        representation = rearrange(representation, 'b w c h -> b w (c h)')
        # |representation| = (batch_size, 8, 1232 * 4) = (batch_size, width, (channel * height))
        representation = self.eff_linear(representation)
        
        # |representation| = (batch_size, 8, 1024)
        context, _ = self.lstm(representation)
        # |conext| = (batch_size, 8, 1024 * 2)
        
        # three_linear
        # two_linear

        y_hat = context.new_zeros(size=(batch_size, 6, self.config['vocabulary_size']))

        # 첫 번째, 두 번째, 세 번째, 네 번째 글자를 예측함
        for idx in range(6):
            if idx < 4:
                y_hat[:, idx, :] = self.three_linear(context[:,idx:idx+3,:].reshape(batch_size, -1))
            else:
                y_hat[:, idx, :] = self.two_linear(context[:,idx:idx+2,:].reshape(batch_size, -1))
        # |y_hat| = (batch_size, 6, vocabulary_size)
        # llabels| = (batch_size, 6)
        loss, acc = self.compute_loss(y_hat, labels, is_valid = False)

        metrics = {'train_loss':loss}
        self.log_dict(metrics, prog_bar=True)
        return {
            "loss":loss
        }

    def validation_step(self, batch, batch_idx):
       # batch = (image, label)
        # |image| = (batch_size, channel, h, w)
        # <수정전>
        # |label| = ("안녕하세요", "나는") <- tuple 형태로 담겨져 있음
        # <수정후>
        # |label| = (batch_size, 6)
        images, labels = batch
        batch_size = images.shape[0]

        representation = self.eff_feature_extract(images)
        # |representation| = (batch_size,1232, 4, 8) = (batch_size, channel, height, width)
        representation = representation.permute(0, 3, 1, 2)
        representation = rearrange(representation, 'b w c h -> b w (c h)')
        # |representation| = (batch_size, 8, 1232 * 4) = (batch_size, width, (channel * height))
        representation = self.eff_linear(representation)
        
        # |representation| = (batch_size, 8, 1024)
        context, _ = self.lstm(representation)
        # |conext| = (batch_size, 8, 1024 * 2)
        
        # three_linear
        # two_linear

        y_hat = context.new_zeros(size=(batch_size, 6, self.config['vocabulary_size']))

        # 첫 번째, 두 번째, 세 번째, 네 번째 글자를 예측함
        for idx in range(6):
            if idx < 4:
                y_hat[:, idx, :] = self.three_linear(context[:,idx:idx+3,:].reshape(batch_size, -1))
            else:
                y_hat[:, idx, :] = self.two_linear(context[:,idx:idx+2,:].reshape(batch_size, -1))
        # |y_hat| = (batch_size, 6, vocabulary_size)
        # llabels| = (batch_size, 6)
        loss, acc = self.compute_loss(y_hat, labels, is_valid = True)


        metrics = {'val_loss':loss, 'val_acc':acc}
        self.log_dict(metrics, prog_bar=True)

        return {
            "loss":loss
        }
    def test_step(self, batch, batch_idx):
        pass
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        # batch = (image, label)
        # |image| = (batch_size, channel, h, w)
        # <수정전>
        # |label| = ("안녕하세요", "나는") <- tuple 형태로 담겨져 있음
        # <수정후>
        # |label| = (batch_size, 6)
        images = batch
        batch_size = images.shape[0]

        representation = self.eff_feature_extract(images)
        # |representation| = (batch_size,1232, 4, 8) = (batch_size, channel, height, width)
        representation = representation.permute(0, 3, 1, 2)
        representation = rearrange(representation, 'b w c h -> b w (c h)')
        # |representation| = (batch_size, 8, 1232 * 4) = (batch_size, width, (channel * height))
        representation = self.eff_linear(representation)
        # |representation| = (batch_size, 8, 1024)

        context, _ = self.lstm(representation)
        # |conext| = (batch_size, 8, 1024 * 2)
        
        # three_linear
        # two_linear

        y_hat = context.new_zeros(size=(batch_size, 6, self.config['vocabulary_size']))

        # 첫 번째, 두 번째, 세 번째, 네 번째 글자를 예측함
        for idx in range(6):
            if idx < 4:
                # |context[:,idx:idx+3,:]| = (batch_size, 3, 2048)
                y_hat[:, idx, :] = self.three_linear(rearrange(context[:,idx:idx+3,:], 'b t h -> b (t h)'))
            else:
                y_hat[:, idx, :] = self.two_linear(rearrange(context[:,idx:idx+2,:], 'b t h -> b (t h)'))

        return y_hat

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

## loc으로 꺼내온 데이터 에는 reset_index를 해줘야함

reset_index 해줘야 하는거 모르고 개고생했네 ....

In [21]:
skf = StratifiedKFold(n_splits = config['n_splits'], random_state = config['random_seed'], shuffle=True)

In [22]:
from pytorch_lightning.callbacks import RichProgressBar, EarlyStopping, ModelCheckpoint

for k_fold, (train_index, valid_index) in enumerate(skf.split(np.zeros(len(train_df)), train_df['label_len'])):
    if k_fold <= 6:
        continue
    train_dataset = TextDataset(train_df.loc[train_index, 'img_path'].reset_index(drop=True), train_df.loc[train_index, 'label'].reset_index(drop=True), is_train = True)
    train_dataloader = DataLoader(train_dataset, batch_size = config['batch_size'], collate_fn = TextCollator(is_train = True), shuffle = True)

    valid_dataset = TextDataset(train_df.loc[valid_index, 'img_path'].reset_index(drop=True), train_df.loc[valid_index, 'label'].reset_index(drop=True), is_valid = True)
    valid_dataloader = DataLoader(valid_dataset, batch_size = config['batch_size'], collate_fn = TextCollator(is_train = True))

    dirpath = f"/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/fold{k_fold}"
    checkpoint_callback = ModelCheckpoint(
    dirpath=dirpath,
    save_last = True,
    save_top_k =-1,
    filename='{epoch}-{step}-{train_loss:.4f}-{train_acc}-{val_loss:.4f}-{val_acc}',
    verbose=True,
    monitor='val_acc',
    mode='max'
    )
    
    model = OCRModel(config)
    trainer = pl.Trainer(max_epochs = config['max_epochs'], accelerator="gpu", accumulate_grad_batches = config['accumulate_grad_batches'], precision=16,
                         callbacks=[EarlyStopping('val_acc', patience = config['patience'], mode='max', verbose = True), checkpoint_callback])
    # trainer = pl.Trainer(max_epochs = config['max_epochs'], accelerator="cpu", accumulate_grad_batches = config['accumulate_grad_batches'],
    #                      callbacks=[EarlyStopping('val_acc', patience = config['patience'], mode='max', verbose = True), checkpoint_callback])
    trainer.fit(model, train_dataloader, valid_dataloader)
    # trainer.validate(model, valid_dataloader)
    
    del train_dataset, train_dataloader, valid_dataset, valid_dataloader, model, trainer

    gc.collect()
    torch.cuda.empty_cache() 

Downloading: "https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth" to /root/.cache/torch/hub/checkpoints/regnety_160-a5fe301d.pth
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type             | Params
---------------------------------------------------------
0 | eff_feature_extract | Sequential       | 59.0 M
1 | eff_linear          | Sequential       | 5.0 M 
2 | lstm                | LSTM             | 16.8 M
3 | three_linear        | Linear           | 1

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_acc improved. New best score: 0.000
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 68: 'val_acc' reached 0.00013 (best 0.00013), saving model to '/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/fold7/epoch=0-step=68-train_loss=1.8728-train_acc=0-val_loss=1.9545-val_acc=0.00013005592404734037.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 31, global step 2176: 'val_acc' reached 0.96241 (best 0.96826), saving model to '/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/fold9/epoch=31-step=2176-train_loss=0.0204-train_acc=0-val_loss=0.0315-val_acc=0.9624089490114464.ckpt' as top 32


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 32, global step 2244: 'val_acc' reached 0.95721 (best 0.96826), saving model to '/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/fold9/epoch=32-step=2244-train_loss=0.0046-train_acc=0-val_loss=0.0353-val_acc=0.9572060353798127.ckpt' as top 33


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Monitored metric val_acc did not improve in the last 5 records. Best score: 0.968. Signaling Trainer to stop.
INFO:pytorch_lightning.utilities.rank_zero:Epoch 33, global step 2312: 'val_acc' reached 0.95552 (best 0.96826), saving model to '/content/drive/MyDrive/Colab Notebooks/DACON/2023_교원그룹_AI_챌린지/fold9/epoch=33-step=2312-train_loss=0.0081-train_acc=0-val_loss=0.0358-val_acc=0.9555150884495317.ckpt' as top 34
